# Creating the Dataset for Trade Openness Analysis

This is an assignment for the first week of the "Regression Modeling in Practice" course by Wesleyan University: "Writing About Your Data". 

#### Sample

The dataset includes economical and geographical indicators for 217 countries for the 2017 — 2019.    

#### Procedure

Majority of **the World Bank Indicators** are based on data originally collected, compiled and published by other sources, including other international organizations such as UN specialized agencies (sometimes in cooperation with the World Bank), national statistical offices, organizations with a specific research or monitoring focus, the private sector, and academic studies. The data from external sources is subject to vetting and curation processes, including a range of checks and quality control efforts from the WDI team. However, many indicators, such as 'Ease of Doing Business' calculated directly from data gathered through specific surveys carried out locally in-country. <a href='https://datatopics.worldbank.org/world-development-indicators/stories/world-development-indicators-the-story.html'>Source</a>

**WTO Data** on bound tariffs based on concessions on goods that a WTO Member made in trade negotiations, or negotiated during its accession to the WTO. Data on applied tariffs based on reports of Member countries, they are obligated to notify WTO on applied tariffs yearly. Applied tariffs are also notified by non-WTO Member countries that are currently negotiating their accession to the WTO. <a href='https://timeseries.wto.org/assets/UserGuide/TechnicalNotes_en.pdf'>Source</a>

#### Measures

World Bank Data:
- <code>gdp</code> Gross Domestic Product of a country in current US dollars. 
- <code>gpd_ppc</code> Gross Domestic Product per Capita: gross domestic product of a country divided by its total population. Indicates the income level of the country.  
- <code>imports</code> Imports of goods and services in current US dollars
- <code>exports</code> Exports of goods and services in current US dollars. 

Calculations for exports and all of the above indicators are based on World Bank national accounts data, and OECD National Accounts data files.

- <code>population</code> Number of people living in the country. Calculations based on:
     1. United Nations Population Division. World Population Prospects: 2019 Revision. 
     2. Census reports and other statistical publications from national statistical offices
     3. Eurostat: Demographic Statistics, 
     4. United Nations Statistical Division. Population and Vital Statistics Reprot (various years), 
     5. U.S. Census Bureau: International Database, 
     6. Secretariat of the Pacific Community: Statistics and Demography Programme.
     

- <code>business_ease</code> Ease of doing business score ranging from 0 for lowest performance to 100 for best performance. Calculated by the World Bank, Doing Business project.
- <code>region</code> Categorical variables coded with regions of the world. Contains 7 regions: 'Latin America & Caribbean ', 'South Asia', 'Sub-Saharan Africa ', 'Europe & Central Asia', 'Middle East & North Africa', 'East Asia & Pacific', 'North America'.
- <code>code</code> Officially assigned ISO 3166-1 alpha-3 codes, using the English short country names officially used by the ISO 3166 Maintenance Agency. 

WTO Data:
- <code>tariff</code> Applied Most Favourite Nation Tariffs (Percentage), as notified by countries to the WTO.
- <code>WTO status</code> Relationship of the country with WTO. Categorical variable with tree options: 'Member', 'Observer', and 'No participation'.

In [3]:
#import libraries
import pandas as pd
import numpy as np
from pandas_datareader import wb

## Data Gathering

### GPD, Import & Export using World Bank API

#### Step 1. Find indicators

In [414]:
#wb.search(string='iso', field='name') 
#wb.search(string='GDP per capita', field='name') #to find GDP indicators
#wb.search(string='Economy & Growth ; Trade', field='topics') #to find import and export indicators
#wb.search(string='Population, total', field='name') #to find population indicators

#### Problem 1. Wrong indicator

In [415]:
wb.search(string='GDP', field='name').head(2) 

,id,name,source,sourceNote,sourceOrganization,topics,unit
675,5.51.01.10.gdp,Per capita GDP growth,Statistical Capacity Indicators,GDP per capita is the sum of gross value added...,b'World Development Indicator (WDI) databank. ...,,
677,6.0.GDP_current,GDP (current $),LAC Equity Lab,GDP is the sum of gross value added by all res...,b'World Development Indicators (World Bank)',Economy & Growth,


In [416]:
deu = wb.download(country='DEU', indicator='6.0.GDP_current', start=1998, end=2018)
deu.head()

/Users/lucy/anaconda3/lib/python3.7/site-packages/pandas_datareader/wb.py:659: UserWarning: No results found from world bank. Indicator: 6.0.GDP_current
  warnings.warn(msg)


ValueError: No indicators returned data.

##### Solution

Let's try Google:
<img src="google.png" width="600">

In [275]:
deu = wb.download(country='DEU', indicator='NY.GDP.MKTP.CD', start=2018, end=2018)
deu.head()

,,NY.GDP.MKTP.CD
country,year,
Germany,2018,3.947620e+12


#### Problem 2. Regional aggregates in countries

In [276]:
data = wb.download(country='all', indicator='NY.GDP.PCAP.CD', start=2018, end=2018)
data.head()

,,NY.GDP.PCAP.CD
country,year,
Arab World,2018,6610.150262
Caribbean small states,2018,9991.016149
Central Europe and the Baltics,2018,15926.098238
Early-demographic dividend,2018,3582.136212
East Asia & Pacific,2018,11142.551874


##### Solution

In [417]:
countries = wb.get_countries()
countries.head()

,adminregion,capitalCity,iso3c,incomeLevel,iso2c,latitude,lendingType,longitude,name,region
0,,Oranjestad,ABW,High income,AW,12.51670,Not classified,-70.0167,Aruba,Latin America & Caribbean
1,South Asia,Kabul,AFG,Low income,AF,34.52280,IDA,69.1761,Afghanistan,South Asia
2,,,AFR,Aggregates,A9,NaN,Aggregates,NaN,Africa,Aggregates
3,Sub-Saharan Africa (excluding high income),Luanda,AGO,Lower middle income,AO,-8.81155,IBRD,13.2420,Angola,Sub-Saharan Africa
4,Europe & Central Asia (excluding high income),Tirane,ALB,Upper middle income,AL,41.33170,IBRD,19.8172,Albania,Europe & Central Asia


In [418]:
# Get names of indexes for Regions
index_names = countries[countries['incomeLevel'] == 'Aggregates'].index

# Delete these row indexes from dataFrame
countries.drop(index_names, inplace=True)
countries.head()

,adminregion,capitalCity,iso3c,incomeLevel,iso2c,latitude,lendingType,longitude,name,region
0,,Oranjestad,ABW,High income,AW,12.51670,Not classified,-70.0167,Aruba,Latin America & Caribbean
1,South Asia,Kabul,AFG,Low income,AF,34.52280,IDA,69.1761,Afghanistan,South Asia
3,Sub-Saharan Africa (excluding high income),Luanda,AGO,Lower middle income,AO,-8.81155,IBRD,13.2420,Angola,Sub-Saharan Africa
4,Europe & Central Asia (excluding high income),Tirane,ALB,Upper middle income,AL,41.33170,IBRD,19.8172,Albania,Europe & Central Asia
5,,Andorra la Vella,AND,High income,AD,42.50750,Not classified,1.5218,Andorra,Europe & Central Asia


In [419]:
codes = np.array(countries['iso3c'].unique()).tolist()

In [420]:
data = wb.download(country=codes, indicator=['NY.GDP.MKTP.CD','NY.GDP.PCAP.CD','NE.IMP.GNFS.CD','NE.EXP.GNFS.CD','SP.POP.TOTL','IC.BUS.DFRN.XQ'], start=2017, end=2019)
data.head()

/Users/lucy/anaconda3/lib/python3.7/site-packages/pandas_datareader/wb.py:593: UserWarning: Non-standard ISO country codes: CHI, XKX
  "Non-standard ISO " "country codes: %s" % tmp, UserWarning


NY.GDP.MKTP.CD  NY.GDP.PCAP.CD  NE.IMP.GNFS.CD  \
country     year                                                   
Aruba       2019             NaN             NaN             NaN   
            2018             NaN             NaN             NaN   
            2017    2.700559e+09    25630.266492    2.031844e+09   
Afghanistan 2019             NaN             NaN             NaN   
            2018    1.936297e+10      520.896603             NaN   

                  NE.EXP.GNFS.CD  SP.POP.TOTL  IC.BUS.DFRN.XQ  
country     year                                               
Aruba       2019             NaN          NaN             NaN  
            2018             NaN     105845.0             NaN  
            2017    1.912291e+09     105366.0             NaN  
Afghanistan 2019             NaN          NaN        44.06497  
            2018             NaN   37172386.0        44.20343

In [421]:
data.columns = ['gdp','gdp_ppc','imports','exports','population','business_ease']
data.reset_index(inplace=True) 

### Get country information

In [423]:
countries = countries[['iso3c','name','region']]
countries.columns = ['code','country','region']
countries.head()

,code,country,region
0,ABW,Aruba,Latin America & Caribbean
1,AFG,Afghanistan,South Asia
3,AGO,Angola,Sub-Saharan Africa
4,ALB,Albania,Europe & Central Asia
5,AND,Andorra,Europe & Central Asia


In [424]:
data = data.merge(countries, how='left', left_on='country', right_on='country')
data.head()

,country,year,gdp,gdp_ppc,imports,exports,population,business_ease,code,region
0,Aruba,2019,NaN,NaN,NaN,NaN,NaN,NaN,ABW,Latin America & Caribbean
1,Aruba,2018,NaN,NaN,NaN,NaN,105845.0,NaN,ABW,Latin America & Caribbean
2,Aruba,2017,2.700559e+09,25630.266492,2.031844e+09,1.912291e+09,105366.0,NaN,ABW,Latin America & Caribbean
3,Afghanistan,2019,NaN,NaN,NaN,NaN,NaN,44.06497,AFG,South Asia
4,Afghanistan,2018,1.936297e+10,520.896603,NaN,NaN,37172386.0,44.20343,AFG,South Asia


In [346]:
print('2019')
data_2019 = data.loc[data['year'] == '2019']
print(data_2019.info())
print('____')
print('2018')
data_2018 = data.loc[data['year'] == '2018']
print(data_2018.info())
print('____')
print('2017')
data_2017 = data.loc[data['year'] == '2017']
print(data_2017.info())

2019
<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 648
Data columns (total 10 columns):
country         217 non-null object
year            217 non-null object
gdp             0 non-null float64
gdp_ppc         0 non-null float64
imports         0 non-null float64
exports         0 non-null float64
population      0 non-null float64
code            217 non-null object
income_level    217 non-null object
region          217 non-null object
dtypes: float64(5), object(5)
memory usage: 18.6+ KB
None
____
2018
<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 1 to 649
Data columns (total 10 columns):
country         217 non-null object
year            217 non-null object
gdp             195 non-null float64
gdp_ppc         195 non-null float64
imports         169 non-null float64
exports         169 non-null float64
population      216 non-null float64
code            217 non-null object
income_level    217 non-null object
region          217 non-null obje

The entries for 2017 are the most full, so we will analyse data from 2017.

In [425]:
data = data.loc[data['year'] == '2017']

In [426]:
data.sample(10)

,country,year,gdp,gdp_ppc,imports,exports,population,business_ease,code,region
494,Romania,2017,2.116954e+11,10807.684485,9.228664e+10,8.779028e+10,1.958749e+07,73.04132,ROU,Europe & Central Asia
269,India,2017,2.652243e+12,1981.268706,5.831239e+11,4.981074e+11,1.338659e+09,60.90760,IND,South Asia
395,Montenegro,2017,4.844592e+09,7784.065290,3.124751e+09,1.988957e+09,6.223730e+05,73.84108,MNE,Europe & Central Asia
506,Sudan,2017,1.230534e+11,3015.024430,1.453916e+10,1.192612e+10,4.081340e+07,45.00301,SDN,Sub-Saharan Africa
65,Bosnia and Herzegovina,2017,1.808012e+10,5394.591220,1.020019e+10,7.245880e+09,3.351527e+06,65.24652,BIH,Europe & Central Asia
398,Mongolia,2017,1.142576e+10,3669.417540,6.561657e+09,6.831989e+09,3.113779e+06,67.40642,MNG,East Asia & Pacific
122,"Congo, Rep.",2017,8.701335e+09,1702.571349,5.670695e+09,8.182173e+09,5.110702e+06,37.86014,COG,Sub-Saharan Africa
560,Syrian Arab Republic,2017,NaN,NaN,NaN,NaN,1.706800e+07,41.44563,SYR,Middle East & North Africa
5,Afghanistan,2017,2.019176e+10,556.302139,9.153344e+09,1.192286e+09,3.629640e+07,37.13062,AFG,South Asia
311,Kiribati,2017,1.855725e+08,1625.575970,1.707541e+08,2.452483e+07,1.141580e+05,46.16176,KIR,East Asia & Pacific


### Membership & Tariffs from WTO Database

In [427]:
tariff = pd.read_csv('wto.csv', sep=';')
tariff.head()

,Indicator Category,Indicator Code,Indicator,Reporting Economy Code,Reporting Economy ISO3A Code,Reporting Economy,Partner Economy Code,Partner Economy ISO3A Code,Partner Economy,Product/Sector Classification Code,...,Period,Frequency Code,Frequency,Unit Code,Unit,Year,Value Flag Code,Value Flag,Text Value,Value
0,MFN - All products,TP_A_0030,MFN - Trade weighted average duty,4,AFG,Afghanistan,-,NaN,-,-,...,Annual,A,Annual,PCT,Percent,2008,NaN,NaN,NaN,8.053204
1,MFN - All products,TP_A_0030,MFN - Trade weighted average duty,4,AFG,Afghanistan,-,NaN,-,-,...,Annual,A,Annual,PCT,Percent,2012,NaN,NaN,NaN,8.313539
2,MFN - All products,TP_A_0030,MFN - Trade weighted average duty,4,AFG,Afghanistan,-,NaN,-,-,...,Annual,A,Annual,PCT,Percent,2013,NaN,NaN,NaN,6.806913
3,MFN - All products,TP_A_0030,MFN - Trade weighted average duty,4,AFG,Afghanistan,-,NaN,-,-,...,Annual,A,Annual,PCT,Percent,2018,NaN,NaN,NaN,6.071134
4,MFN - All products,TP_A_0030,MFN - Trade weighted average duty,8,ALB,Albania,-,NaN,-,-,...,Annual,A,Annual,PCT,Percent,2006,NaN,NaN,NaN,6.644176


In [428]:
tariff = tariff[['Reporting Economy ISO3A Code','Year','Value']]
tariff.columns = ['code','year','tariff']
tariff.head()

,code,year,tariff
0,AFG,2008,8.053204
1,AFG,2012,8.313539
2,AFG,2013,6.806913
3,AFG,2018,6.071134
4,ALB,2006,6.644176


In [429]:
def add_missing_years(grp):
    _ = grp.set_index('year')
    _ = _.reindex(list(range(2005,2018)))
    del _['code']
    return _

# Group by country and extend
tariff = tariff.groupby('code').apply(add_missing_years)
tariff = tariff.reset_index()
tariff.head()

,code,year,tariff
0,AFG,2005,NaN
1,AFG,2006,NaN
2,AFG,2007,NaN
3,AFG,2008,8.053204
4,AFG,2009,NaN


In [430]:
tariff = tariff.groupby('code').apply(lambda x: x.fillna(method='ffill'))
tariff.head(15)

,code,year,tariff
0,AFG,2005,NaN
1,AFG,2006,NaN
2,AFG,2007,NaN
3,AFG,2008,8.053204
4,AFG,2009,8.053204
5,AFG,2010,8.053204
6,AFG,2011,8.053204
7,AFG,2012,8.313539
8,AFG,2013,6.806913
9,AFG,2014,6.806913


In [431]:
tariff['year'] = tariff['year'].astype('int')
data['year'] = data['year'].astype('int')

In [434]:
data = data.merge(tariff, how='left', left_on=['code','year'], right_on=['code','year'])

In [435]:
data.head()

,country,year,gdp,gdp_ppc,imports,exports,population,business_ease,code,region,tariff
0,Aruba,2017,2.700559e+09,25630.266492,2.031844e+09,1.912291e+09,105366.0,NaN,ABW,Latin America & Caribbean,NaN
1,Afghanistan,2017,2.019176e+10,556.302139,9.153344e+09,1.192286e+09,36296400.0,37.13062,AFG,South Asia,6.806913
2,Angola,2017,1.221238e+11,4095.812942,2.839711e+10,3.542092e+10,29816748.0,39.00774,AGO,Sub-Saharan Africa,9.375700
3,Albania,2017,1.302506e+10,4532.890162,6.070288e+09,4.110274e+09,2873457.0,66.83770,ALB,Europe & Central Asia,3.328795
4,Andorra,2017,3.013387e+09,39134.393371,NaN,NaN,77001.0,NaN,AND,Europe & Central Asia,NaN


In [436]:
memb = pd.read_csv('mem-wto.csv', sep=';')
memb.drop('country', axis=1,inplace=True)
memb.head()

,wto_status,code
0,Member,AFG
1,Member,ALB
2,Member,AGO
3,Member,ATG
4,Member,ARG


In [437]:
data = data.merge(memb, how='left', left_on='code', right_on='code')
data.head()

,country,year,gdp,gdp_ppc,imports,exports,population,business_ease,code,region,tariff,wto_status
0,Aruba,2017,2.700559e+09,25630.266492,2.031844e+09,1.912291e+09,105366.0,NaN,ABW,Latin America & Caribbean,NaN,NaN
1,Afghanistan,2017,2.019176e+10,556.302139,9.153344e+09,1.192286e+09,36296400.0,37.13062,AFG,South Asia,6.806913,Member
2,Angola,2017,1.221238e+11,4095.812942,2.839711e+10,3.542092e+10,29816748.0,39.00774,AGO,Sub-Saharan Africa,9.375700,Member
3,Albania,2017,1.302506e+10,4532.890162,6.070288e+09,4.110274e+09,2873457.0,66.83770,ALB,Europe & Central Asia,3.328795,Member
4,Andorra,2017,3.013387e+09,39134.393371,NaN,NaN,77001.0,NaN,AND,Europe & Central Asia,NaN,Observer


In [438]:
data['wto_status'].fillna('No perticipation',inplace=True)
data.head()

,country,year,gdp,gdp_ppc,imports,exports,population,business_ease,code,region,tariff,wto_status
0,Aruba,2017,2.700559e+09,25630.266492,2.031844e+09,1.912291e+09,105366.0,NaN,ABW,Latin America & Caribbean,NaN,No perticipation
1,Afghanistan,2017,2.019176e+10,556.302139,9.153344e+09,1.192286e+09,36296400.0,37.13062,AFG,South Asia,6.806913,Member
2,Angola,2017,1.221238e+11,4095.812942,2.839711e+10,3.542092e+10,29816748.0,39.00774,AGO,Sub-Saharan Africa,9.375700,Member
3,Albania,2017,1.302506e+10,4532.890162,6.070288e+09,4.110274e+09,2873457.0,66.83770,ALB,Europe & Central Asia,3.328795,Member
4,Andorra,2017,3.013387e+09,39134.393371,NaN,NaN,77001.0,NaN,AND,Europe & Central Asia,NaN,Observer


### Landlocked Countries

In [439]:
landlocked = pd.read_csv('landlocked.csv', sep=';')
landlocked['landlocked'] = landlocked['landlocked'].astype('int')
landlocked.drop('country', axis=1, inplace=True)
landlocked.columns = ['landlocked','code']
landlocked.head()

,landlocked,code
0,1,AFG
1,1,AND
2,1,ARM
3,1,NaN
4,1,AUT


In [440]:
data = data.merge(landlocked, how='left', left_on='code', right_on='code')

In [441]:
data.landlocked.fillna(0, inplace=True)
data['landlocked'] = data['landlocked'].astype('int')
data.head()

,country,year,gdp,gdp_ppc,imports,exports,population,business_ease,code,region,tariff,wto_status,landlocked
0,Aruba,2017,2.700559e+09,25630.266492,2.031844e+09,1.912291e+09,105366.0,NaN,ABW,Latin America & Caribbean,NaN,No perticipation,0
1,Afghanistan,2017,2.019176e+10,556.302139,9.153344e+09,1.192286e+09,36296400.0,37.13062,AFG,South Asia,6.806913,Member,1
2,Angola,2017,1.221238e+11,4095.812942,2.839711e+10,3.542092e+10,29816748.0,39.00774,AGO,Sub-Saharan Africa,9.375700,Member,0
3,Albania,2017,1.302506e+10,4532.890162,6.070288e+09,4.110274e+09,2873457.0,66.83770,ALB,Europe & Central Asia,3.328795,Member,0
4,Andorra,2017,3.013387e+09,39134.393371,NaN,NaN,77001.0,NaN,AND,Europe & Central Asia,NaN,Observer,1


### Openness

In [442]:
data['openness'] = (data['imports'] + data['exports']) / data['gdp']
data.head()

,country,year,gdp,gdp_ppc,imports,exports,population,business_ease,code,region,tariff,wto_status,landlocked,openness
0,Aruba,2017,2.700559e+09,25630.266492,2.031844e+09,1.912291e+09,105366.0,NaN,ABW,Latin America & Caribbean,NaN,No perticipation,0,1.460488
1,Afghanistan,2017,2.019176e+10,556.302139,9.153344e+09,1.192286e+09,36296400.0,37.13062,AFG,South Asia,6.806913,Member,1,0.512369
2,Angola,2017,1.221238e+11,4095.812942,2.839711e+10,3.542092e+10,29816748.0,39.00774,AGO,Sub-Saharan Africa,9.375700,Member,0,0.522568
3,Albania,2017,1.302506e+10,4532.890162,6.070288e+09,4.110274e+09,2873457.0,66.83770,ALB,Europe & Central Asia,3.328795,Member,0,0.781613
4,Andorra,2017,3.013387e+09,39134.393371,NaN,NaN,77001.0,NaN,AND,Europe & Central Asia,NaN,Observer,1,NaN


## Download the CSV File

In [444]:
data.to_csv(r'openness_df.csv', index = False)